In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [7]:
with open(r'C:\Users\Dell\Downloads\book\sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [11]:
len(tokenizer.word_index)

8199

In [12]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [21]:
max_sequence_len=max([len(seq) for seq in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [25]:
input_sequences[0:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    1, 1561],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    1, 1561,    5]])

In [26]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [27]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1, 1561],
       [   0,    0,    0, ...,    1, 1561,    5],
       ...,
       [   0,    0,    0, ...,   28,    1, 8198],
       [   0,    0,    0, ...,    1, 8198, 8199],
       [   0,    0,    0, ..., 8198, 8199, 3187]])

In [32]:
y[0:4]

array([1561,    5,  129,   34])

In [33]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [34]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 100)           820000    
_________________________________________________________________
lstm (LSTM)                  (None, 150)               150600    
_________________________________________________________________
dense (Dense)                (None, 8200)              1238200   
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 132s 43ms/step - loss: 6.2342 - accuracy: 0.0761
Epoch 2/100
3010/3010 [==============================] - 124s 41ms/step - loss: 5.5043 - accuracy: 0.1242
Epoch 3/100
3010/3010 [==============================] - 120s 40ms/step - loss: 5.1195 - accuracy: 0.1470
Epoch 4/100
3010/3010 [==============================] - 120s 40ms/step - loss: 4.7944 - accuracy: 0.1655
Epoch 5/100
3010/3010 [==============================] - 133s 44ms/step - loss: 4.4936 - accuracy: 0.1813
Epoch 6/100
3010/3010 [==============================] - 121s 40ms/step - loss: 4.2135 - accuracy: 0.2006
Epoch 7/100
3010/3010 [==============================] - 132s 44ms/step - loss: 3.9497 - accuracy: 0.2248s - loss: 3.9497 - accuracy
Epoch 8/100
3010/3010 [==============================] - 119s 39ms/step - loss: 3.6970 - accuracy: 0.2522
Epoch 9/100
3010/3010 [==============================] - 117s 39ms/step - loss: 3.4584 - accuracy: 0.2851
Epoch 10/100
3010/3

3010/3010 [==============================] - 122s 41ms/step - loss: 0.5388 - accuracy: 0.8638
Epoch 77/100
3010/3010 [==============================] - 119s 39ms/step - loss: 0.5366 - accuracy: 0.8639
Epoch 78/100
3010/3010 [==============================] - 121s 40ms/step - loss: 0.5295 - accuracy: 0.8661
Epoch 79/100
3010/3010 [==============================] - 120s 40ms/step - loss: 0.5332 - accuracy: 0.8644
Epoch 80/100
3010/3010 [==============================] - 120s 40ms/step - loss: 0.5300 - accuracy: 0.8647
Epoch 81/100
3010/3010 [==============================] - 125s 42ms/step - loss: 0.5259 - accuracy: 0.8663
Epoch 82/100
3010/3010 [==============================] - 122s 40ms/step - loss: 0.5250 - accuracy: 0.8653
Epoch 83/100
3010/3010 [==============================] - 122s 40ms/step - loss: 0.5251 - accuracy: 0.8657
Epoch 84/100
3010/3010 [==============================] - 129s 43ms/step - loss: 0.5209 - accuracy: 0.8660
Epoch 85/100
3010/3010 [==========================

In [36]:
#some vague account of his doings
seed_text = "From time to time i heard"
next_words = 6

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

From time to time i heard what was yourself about to the
